## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/3', creation_time=1684462448847, experiment_id='3', last_update_time=1684462448847, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/2', creation_time=1684461098040, experiment_id='2', last_update_time=1684461098040, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/1', creation_time=1684397621355, experiment_id='1', last_update_time=1684397621355, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/0', creation_time=1684397621348, experiment_id='0', last_update_time=1684397621348, lifecycle_stage='acti

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: '/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/3/50ac983672804fe2abdd01c9dcb4eaf2/artifacts'


/Users/ngothai/opt/anaconda3/envs/torch113/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/3', creation_time=1684462448847, experiment_id='3', last_update_time=1684462448847, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/2', creation_time=1684461098040, experiment_id='2', last_update_time=1684461098040, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/1', creation_time=1684397621355, experiment_id='1', last_update_time=1684397621355, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/0', creation_time=1684397621348, experiment_id='0', last_update_time=1684397621348, lifecycle_stage='acti

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient
client = MlflowClient("http://127.0.0.1:5000")

In [8]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1684461216527, description='', last_updated_timestamp=1684461329377, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1684461216545, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2023-05-19', last_updated_timestamp=1684461339146, name='nyc-taxi-regressor', run_id='7003591f6f724baca4a1822b80bb0825', run_link='', source='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/1/7003591f6f724baca4a1822b80bb0825/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>], name='nyc-taxi-regressor', tags={}>,
 <RegisteredModel: aliases={}, creation_timestamp=1684459763889, description='', last_updated_timestamp=1684460073753, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1684459763920, current_stage='Staging', description='', last_updated_timestamp=1684460073753, name='nyc-taxi-xgboost', run_id='7003591f6f724baca4a18

In [9]:
run_id = client.list_run_infos(experiment_id='3')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

AttributeError: 'MlflowClient' object has no attribute 'list_run_infos'

In [10]:
isris_run = client.search_runs(experiment_ids='3')

In [11]:
isris_run

[<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "50ac983672804fe2abdd01c9dcb4eaf2", '
                              '"artifact_path": "models", "utc_time_created": '
                              '"2023-05-19 02:21:08.533054", "flavors": '
                              '{"python_function": {"model_path": "model.pkl", '
                              '"predict_fn": "predict", "loader_module": '
                              '"mlflow.sklearn", "python_version": "3.9.16", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "sklearn": '
                              '{"pickled_model": "model.pkl", '
                              '"sklearn_version": "1.1.3", '
                              '"serialization_format": "cloudpickle", "code": '
                              'null}}, "model_uuid": '
                              

In [12]:
len(isris_run)

2

In [13]:
run_id = isris_run[0].info.run_id
run_id

'50ac983672804fe2abdd01c9dcb4eaf2'

In [14]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2023/05/19 09:24:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1684463055526, current_stage='None', description='', last_updated_timestamp=1684463055526, name='iris-classifier', run_id='50ac983672804fe2abdd01c9dcb4eaf2', run_link='', source='/Volumes/Data/projects/dl_labs/mlops-zoomcamp-2023/02-experiment-tracking/mlruns/3/50ac983672804fe2abdd01c9dcb4eaf2/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>